<a href="https://colab.research.google.com/github/VidyaGanes/Hackathon/blob/main/Final_Mobile_image_data_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the webpage
url = "https://www.91mobiles.com/realme-mobile-price-list-in-india"

# Send a GET request to the webpage
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')

# Find all image tags
images = soup.find_all('img')

# Lists to store image URLs and captions
image_urls = []
captions = []

# Loop through all image tags
for img in images:
    # Get the image URL
    img_url = img.get('src')
    # Get the image caption (alt text)
    caption = img.get('alt')

    # Append to lists
    image_urls.append(img_url)
    captions.append(caption)

# Create a pandas DataFrame
df = pd.DataFrame({
    'Image URL': image_urls,
    'Caption': captions
})

# Save the DataFrame to a CSV file
df.to_csv('realme_mobiles_images.csv', index=False)

print("Data saved to realme_mobiles_images.csv")

Data saved to realme_mobiles_images.csv


In [ ]:
df.head()

,Image URL,Caption
0,https://sb.scorecardresearch.com/p?c1=2&c2=187...,score
1,https://www.91-img.com/images/logo.gif,91logo
2,None,
3,//www.91-img.com/pictures/162085-v6-realme-gt-...,realme GT 6T price in India
4,None,


In [ ]:
!unzip phone.zip

Archive:  phone.zip
replace phone/i1.webp? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
df = pd.read_csv('/content/realme_mobiles_images.csv')

In [ ]:
import os
import pandas as pd

# Assuming the images are stored in a directory named 'images'
image_folder = '/content/phone'

# List of image filenames from i1 to i10
image_filenames = [f'i{num}.webp' for num in range(1, 11)]

# Create a list of full image paths
image_paths = [os.path.join(image_folder, filename) for filename in image_filenames]

# Create a new DataFrame with the image paths
df_images = pd.DataFrame(image_paths, columns=['image_path'])

# Concatenate the two DataFrames along the columns (axis=1)
df = pd.concat([df, df_images], axis=1)

# Display the combined DataFrame
print(df)

                                           Image URL  \
0  https://sb.scorecardresearch.com/p?c1=2&c2=187...   
1             https://www.91-img.com/images/logo.gif   
3  //www.91-img.com/pictures/162085-v6-realme-gt-...   
5  //www.91-img.com/pictures/161423-v4-realme-p1-...   
7  //www.91-img.com/pictures/162760-v5-realme-gt-...   
9  //www.91-img.com/pictures/154780-v2-realme-12-...   
2                                                NaN   
4                                                NaN   
6                                                NaN   
8                                                NaN   

                             Caption               image_path  
0                              score   /content/phone/i1.webp  
1                             91logo   /content/phone/i2.webp  
3        realme GT 6T price in India   /content/phone/i4.webp  
5           realme P1 price in India   /content/phone/i6.webp  
7         realme GT 6 price in India   /content/phone/i8.webp  

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(6, 3)

In [ ]:
df

,Image URL,Caption,image_path
0,https://sb.scorecardresearch.com/p?c1=2&c2=187...,score,/content/phone/i1.webp
1,https://www.91-img.com/images/logo.gif,91logo,/content/phone/i2.webp
3,//www.91-img.com/pictures/162085-v6-realme-gt-...,realme GT 6T price in India,/content/phone/i4.webp
5,//www.91-img.com/pictures/161423-v4-realme-p1-...,realme P1 price in India,/content/phone/i6.webp
7,//www.91-img.com/pictures/162760-v5-realme-gt-...,realme GT 6 price in India,/content/phone/i8.webp
9,//www.91-img.com/pictures/154780-v2-realme-12-...,realme 12 Pro Plus price in India,/content/phone/i10.webp


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Example preprocessing function
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))  # Resize image to 224x224 pixels
    img_array = img_to_array(img)
    img_array = tf.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.inception_v3.preprocess_input(img_array)
    return img_array

# Preprocess images
df['preprocessed_images'] = df['image_path'].apply(preprocess_image)

In [ ]:
# Load InceptionV3 model pre-trained on ImageNet without the top classification layer
inception_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

def extract_features(image_array):
    features = inception_model.predict(image_array)
    features = tf.reshape(features, (features.shape[0], -1))
    return features

# Extract features from preprocessed images
df['image_features'] = df['preprocessed_images'].apply(extract_features)

1/1 [==============================] - 0s 144ms/step


In [ ]:
df['image_features']

0    ((tf.Tensor(0.0, shape=(), dtype=float32), tf....
1    ((tf.Tensor(0.6505687, shape=(), dtype=float32...
3    ((tf.Tensor(0.0, shape=(), dtype=float32), tf....
5    ((tf.Tensor(0.0, shape=(), dtype=float32), tf....
7    ((tf.Tensor(0.0, shape=(), dtype=float32), tf....
9    ((tf.Tensor(0.026407495, shape=(), dtype=float...
Name: image_features, dtype: object

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Concatenate
import numpy as np

# Extract captions from the data
captions = df['Caption'].values

# Initialize the tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the captions
tokenizer.fit_on_texts(captions)

# Convert captions to sequences of integers
sequences = tokenizer.texts_to_sequences(captions)

# Get the vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding token

# Determine the maximum length of captions
max_caption_length = max(len(seq) for seq in sequences)

# Pad the sequences
padded_sequences = pad_sequences(sequences, maxlen=max_caption_length, padding='post')

# Function to create sequences of image features and corresponding captions
def create_sequences(image_features, captions, tokenizer, max_length):
    X1, X2, y = [], [], []

    for img_feat, caption in zip(image_features, captions):
        seq = tokenizer.texts_to_sequences([caption])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)
            X1.append(img_feat)
            X2.append(in_seq)
            y.append(out_seq)

    return np.array(X1), np.array(X2), np.array(y)

# Prepare input-output pairs
image_features = np.array(df['image_features'].tolist())
captions = df['Caption'].values

# Ensure image_features has the correct shape
image_features = image_features.reshape((-1, 51200))  # Adjust shape if needed

X1, X2, y = create_sequences(image_features, captions, tokenizer, max_caption_length)

# Build the Image Captioning Model
# Encoder model
image_features_input = Input(shape=(51200,))
image_features_dropout = Dropout(0.5)(image_features_input)
image_features_dense = Dense(256, activation='relu')(image_features_dropout)

# Decoder model
caption_input = Input(shape=(max_caption_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=256)(caption_input)
lstm_layer = LSTM(256)(embedding_layer)
dense_layer = Dense(256, activation='relu')(lstm_layer)

# Combined model
combined_input = Concatenate()([image_features_dense, dense_layer])
output = Dense(vocab_size, activation='softmax')(combined_input)
model = Model(inputs=[image_features_input, caption_input], outputs=output)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = 'accuracy')

In [ ]:
# Check shapes before training
print("X1 shape:", X1.shape)
print("X2 shape:", X2.shape)
print("y shape:", y.shape)

# Reshape y to (batch_size, vocab_size)
y = y.reshape((y.shape[0], vocab_size))

# Check shape of y after reshaping
print("y reshaped shape:", y.shape)


X1 shape: (20, 51200)
X2 shape: (20, 7)
y shape: (20, 1, 14)
y reshaped shape: (20, 14)


In [ ]:
# Train the model
model.fit([X1, X2], y, epochs=30, batch_size=64)

Epoch 1/30
1/1 [==============================] - 3s 3s/step - loss: 3.4286 - accuracy: 0.0500
Epoch 2/30
1/1 [==============================] - 0s 266ms/step - loss: 21.0403 - accuracy: 0.2000
Epoch 3/30
1/1 [==============================] - 0s 327ms/step - loss: 35.5991 - accuracy: 0.2000
Epoch 4/30
1/1 [==============================] - 0s 315ms/step - loss: 38.6644 - accuracy: 0.2000
Epoch 5/30
1/1 [==============================] - 0s 422ms/step - loss: 34.6879 - accuracy: 0.2000
Epoch 6/30
1/1 [==============================] - 1s 515ms/step - loss: 29.8745 - accuracy: 0.2000
Epoch 7/30
1/1 [==============================] - 0s 451ms/step - loss: 25.0488 - accuracy: 0.1500
Epoch 8/30
1/1 [==============================] - 0s 429ms/step - loss: 25.2228 - accuracy: 0.2000
Epoch 9/30
1/1 [==============================] - 0s 413ms/step - loss: 23.9512 - accuracy: 0.2000
Epoch 10/30
1/1 [==============================] - 0s 363ms/step - loss: 18.8037 - accuracy: 0.2000
Epoch 11/30
1

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 7)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 51200)]              0         []                            
                                                                                                  
 embedding (Embedding)       (None, 7, 256)               3584      ['input_3[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 51200)                0         ['input_2[0][0]']             
                                                                                              